# Point In Face

In [ ]:
import uxarray as ux
import numpy as np
from uxarray.grid.utils import _get_cartesian_face_edge_nodes
from uxarray.grid.geometry import point_in_face

grid_path = "../../test/meshfiles/ugrid/quad-hexagon/grid.nc"
grid = ux.open_grid(grid_path)

The function `point_in_face` determines whether or not a given face contains a point. The function parameters are as follows:

* `edges_xyz` are the supplied edges of the face, with shape [n_edges, 2]
* `point_xyz` the cartesian point which is being checked. The dtype should be `np.float64`
* `inclusive` bool flag indicating whether to classify a point that lies on either the edges or nodes as within the face

We can take a face and check to see if it's face center lies within the face.

In [ ]:
# Get the face edge arrays of each face in the grid
faces_edge_cartesian = _get_cartesian_face_edge_nodes(
    grid.face_node_connectivity.values,
    grid.n_face,
    grid.n_max_face_edges,
    grid.node_x.values,
    grid.node_y.values,
    grid.node_z.values,
)

face_center = np.array(
    [grid.face_x.values[0], grid.face_y.values[0], grid.face_z.values[0]],
    dtype=np.float64,
)
point_in_face(faces_edge_cartesian[0], face_center, inclusive=True)

We can do something similar but with the nodes of the face, to showcase the inclusive flag use.

In [ ]:
face_node = np.array(
    [grid.node_x.values[0], grid.node_y.values[0], grid.node_z.values[0]],
    dtype=np.float64,
)
inclusive = point_in_face(faces_edge_cartesian[0], face_node, inclusive=True)
non_inclusive = point_in_face(faces_edge_cartesian[0], face_node, inclusive=False)

print(
    f"Node is in the face when inclusive is set to True: {inclusive}\nNode is not in the face when inclusive is set to False: {non_inclusive}"
)

# Get Faces Containing Point

Inside UXarray we can also retrieve any faces that contain a given point. By simply passing in a point, the function will return an array that contains the indices of the faces that the point is contained within. A point can be contained within many faces if the point is on a node or an edge. This is because many different faces may use the same node, and an edge will likely have two faces saddling it. This function considers these points as part of the faces that contain that edge or node. If an empty array is returned, this is likely because the grid is a partial and the point is within a region that has no faces. 

In [ ]:
# Face center
grid.get_faces_containing_point(face_center)

In [ ]:
# Node which is apart of 3 different faces
grid.get_faces_containing_point(face_node)

In [ ]:
# Pole point which this partial grid does not cover
random_xyz = np.array([0, 0, 1], dtype=np.float64)
grid.get_faces_containing_point(random_xyz)